# Solar Temperatures

This notebook will utilize prepared data to compute the "Feels Like" temperature.

This metric in weather prediction will add a weight to the high levels of humidity that can make a person lower the temperature in
their thermostat.

My goal for this notebook is to add a new attribute to weather data and use the heat index equation to make a new temperature.

In [19]:
import pandas as pd
import numpy as np

# Load the data
halifaxSolar = pd.read_csv('WeatherData/ColdnHumid/halifax/halifaxSolar/halifaxSolarTotal.csv')

halifaxSolar = halifaxSolar.drop(['Unnamed: 0'], axis=1)
halifaxSolar.head()

,Year,Month,Day,DHI (w/m2),DNI (w/m2),Wind Speed (m/s),Relative Humidity (%),Pressure (mbar)
0,2015,1,1,27,120.0,0.2,75.54,998
1,2015,1,2,30,127.0,0.4,73.16,993
2,2015,1,3,22,223.0,0.4,54.87,1017
3,2015,1,4,15,157.0,0.4,86.75,1001
4,2015,1,5,25,106.0,0.6,81.97,983


When dealing with temperatures in locations where there is a consistent high percentage of humidity, it is advised to use
the **heat index** equation. The **heat index** equation combines air temperature and relative humidity, in shaded areas, to posit a human-perceived equivalent temperature, as how hot it would feel if the humidity were some other value in the shade. 

The version of the [heat index equation](https://www.wpc.ncep.noaa.gov/html/heatindex_equation.shtml) I will use is published from the National Oceanic and Atmospheric Administration (NOAA).

The NOAA supplies two versions of the **heat index equation** that vary between thresholds for both temperatures and relative humidities.

It is recommended that the full analysis of the **heat index equation** starts with Steadman's regression and then tested, to be recomputed with
Rothfusz's regression

In [20]:
from math import sqrt, fabs

def rothfusz_heat_index_regression(temperature, relative_humidity):
    """[Computes the 'feels like' temperature based on Rothfusz's regression]

    Args:
        temperature (float): Temperature in Fahrenheit
        relative_humidity (float): Relative humidity in percentage

    Returns:
        float: Adjusted heat index temperature
    """
    T, RH = temperature, relative_humidity

    added_adjustment = ((RH - 85)/10) * ((87 - T)/5)
    subtracted_adjustment = ((13 - RH)/4) * sqrt(17 - fabs(T - 95) / 17)

    heat_index = (-42.379) + (2.04901523)*(T) + (10.14333127)*(RH) - (0.22475541)*(T)*(RH)
    heat_index = heat_index - (0.00683783)*(T)*(T) - (0.05481717)*(RH)*(RH) + (0.00122874)*(T)*(T)*(RH)
    heat_index = heat_index + (0.00085282)*(T)*(RH)*(RH) - (0.00000199)*(T)*(T)*(RH)*(RH)

    if ((RH < 13) and (80 < T < 112)):
        heat_index -= subtracted_adjustment
    elif ((RH > 85) and (80 < T < 87)):
        heat_index += added_adjustment

    return round(heat_index, 2)

In [21]:
def steadman_heat_index_regression(temperature, relative_humidity):
    """[Computes the 'feels like' temperature based on Steadman's regression]

    Args:
        temperature (float): Temperature in Fahrenheit
        relative_humidity (float): Relative humidity in percentage

    Returns:
        float: Adjusted heat index temperature
    """
    T, RH = temperature, relative_humidity
    heat_index = (0.5) * (T + (61.0) + ((T - 68) * 1.2) + (RH * (0.094)))
    
    return round(heat_index, 2)

In practice, the Steadman's regression is computed first and the result averaged with the temperature.

**If** this heat index value is 80°F or higher, the full regression equation along with any adjustment as described above is applied.

In [22]:
def heat_index(temperature, relative_humidity):
    """[Computes the 'feels like' temperature for usage]

    Args:
        temperature (float): Temperature in Fahrenheit
        relative_humidity (float): Relative humidity in percentage

    Returns:
        float: Adjusted heat index temperature based on NOAA's recommendation
    """
    T, RH = temperature, relative_humidity
    
    temp = steadman_heat_index_regression(temperature=T, relative_humidity=RH)
    steadman = (0.5) * (temp + T)

    rothfusz = rothfusz_heat_index_regression(temperature=T, relative_humidity=RH)

    if (steadman >= 80):
        return rothfusz    